# 다중언어 객체명 인식

## 4.1. 데이터셋

In [1]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names('xtreme')
print(f'XTREME subsets: {len(xtreme_subsets)}')

XTREME subsets: 183


In [2]:
panx_subsets = [s for s in xtreme_subsets if s.startswith('PAN')]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [3]:
# 각 언어를 추적하기 위해 파이썬 defaultdict를 사용.
from collections import defaultdict
from datasets import DatasetDict
from datasets import load_dataset

langs = ['de', 'fr', 'it', 'en']
fraces = [0.629, 0.229, 0.084, 0.059]
# 키가 없는 경우 DatasetDict를 반환
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fraces):
  ds = load_dataset('xtreme', name=f'PAN-X.{lang}')
  for split in ds:
    panx_ch[lang][split] = (
      ds[split]
      .shuffle(seed=0)
      .select(range(int(frac * ds[split].num_rows)))
    )
    # 데이터셋에 의도하지 않은 편향이 들어가지 않도록 shuffle을 사용
    # 데이터셋의 일부만 사용하기 위해 select를 사용


In [4]:
import pandas as pd

samples = pd.DataFrame(
  {lang: [panx_ch[lang]['train'].num_rows] for lang in langs},
  index=['Number of training examples'],
)
print(f"sum fr it en: {samples[['fr', 'it', 'en']].sum().sum()}, de: {samples['de'].sum()}")
samples

sum fr it en: 7440, de: 12580


,de,fr,it,en
Number of training examples,12580,4580,1680,1180


In [5]:
# 제로샷 교차 전이

element = panx_ch['de']['train'][0]
for key, value in element.items():
  print(f'{key}: {value}')

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [6]:
for key, value in panx_ch['de']['train'].features.items():
  print(f'{key}: {value}')

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [7]:
tags = panx_ch['de']['train'].features['ner_tags'].feature
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [8]:
def create_tag_names(batch):
  return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

panx_de = panx_ch['de'].map(create_tag_names)

In [9]:
de_example = panx_de['train'][0]
pd.DataFrame([de_example['tokens'], de_example['ner_tags_str']], ['Tokens', 'Tags'])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [10]:
# 개체명의 빈도 계산

from collections import Counter

split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
  for row in dataset['ner_tags_str']:
    for tag in row:
      if tag.startswith('B'):
        tag_type = tag.split('-')[1]
        split2freqs[split][tag_type] += 1

pd.DataFrame.from_dict(split2freqs, orient='index')
        

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


## 4.2. 다중 언어 트랜스포머

* 다중 언어 트랜스포머는 단일 언어 트랜스포머와 비슷하다.
  * 단순히 여러 언어의 문서로 구성된 뿐이다.


**다중 언어 트랜스포머 모델의 일반적인 3가지 방식**

en: 영어 훈련 데이터에서 미세 튜닝한 다음에 각 언어의 태스트 세트에서 평가

each: 언어별 성능을 측정하기 위해 단일 언어의 테스트 세트에서 미세 튜닝하고 평가

all: 모든 언어의 훈련 데이터를 사용하여 훈련한 다음 각 언어의 테스트 세트에서 평가 


## 4.3. XLM-R 토큰화

* XLM-R은 BERT와 같은 토큰화 방식을 사용한다.
  * 단어를 서브워드로 분할하고, 특수 토큰을 추가한다.
  * BERT와 다른 점은 토큰화에 SentencePiece를 사용한다는 점이다.
    * BERT는 WordPiece를 사용한다.
    * SentencePiece는 BERT의 WordPiece보다 더 성능이 좋다고 한다.
    * SentencePiece는 BPE(Byte Pair Encoding) 알고리즘을 사용한다.
      * BPE는 단어를 서브워드로 분할하는 알고리즘이다.

In [11]:
from transformers import AutoTokenizer

bert_model_name = 'bert-base-cased'
xlmr_model_name = 'xlm-roberta-base'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [12]:
text = "Jack sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

In [13]:
pd.DataFrame([bert_tokens, xlmr_tokens], ['BERT', 'XLM-R'])

,0,1,2,3,4,5,6,7,8,9
BERT,[CLS],Jack,spa,##rrow,loves,New,York,!,[SEP],None
XLM-R,<s>,▁Jack,▁spar,row,▁love,s,▁New,▁York,!,</s>


### 4.5.2. 토큰 불류를 위한 사용자 정의 모델 만들기

In [14]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel, RobertaPreTrainedModel

class RobertaForTokenClassification(RobertaPreTrainedModel):
  config_class = XLMRobertaConfig

  def __init__(self, config):
    print(f"confing class type: {type(config)}")
    super().__init__(config)
    # num_labels: 개체명 태그의 수
    self.num_labels = config.num_labels
    # 모델 바디를 로드
    self.roberta = RobertaModel(config, add_pooling_layer=False)
    # 토큰 분류 헤드를 정의
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    # 가중치 초기화
    self.init_weights()

  def forward(self, input_ids=None, attention_mask=None, token_type=None, labels=None, **kwargs):
    # 모델 바디를 사용해 인코더 표현을 얻음
    outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type, **kwargs)
    # 인코더 표현을 헤드에 통과
    sequence_output = self.droupout(outputs[0])
    logits = self.classifier(sequence_output)
    # 손실을 계산
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntorpyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
      # 모델 출력 객체를 반환
      return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)

### 4.5.3. 사용자 저의 모델 로드

In [15]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {idx: idx for idx, tag in enumerate(tags.names)}

In [16]:
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(xlmr_model_name, num_labels=tags.num_classes, id2label=index2tag, label2id=tag2index)

In [17]:
import torch
from device import get_device
from transformers import XLMRobertaForTokenClassification
device = get_device()
xlmr_model = (XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name, config=xlmr_config).to(device))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
input_ids = xlmr_tokenizer.encode(text, return_tensors='pt')
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=['Tokens', 'Input IDs'])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁spar,row,▁love,s,▁New,▁York,!,</s>
Input IDs,0,21763,27148,15555,5161,7,2356,5753,38,2


In [49]:
outputs = xlmr_model(input_ids.to(device)).logits
predicitons = torch.argmax(outputs, dim=-1)
print(f"시퀀스에 있는 토큰 개수 : {len(xlmr_tokens)}")
print(f"출력 크기: {outputs.shape}")

시퀀스에 있는 토큰 개수 : 10
출력 크기: torch.Size([1, 10, 7])


In [50]:
predicitons[0].cpu().numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [62]:
preds = [tags.names[p] for p in predicitons[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["Tokens","Tags"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁spar,row,▁love,s,▁New,▁York,!,</s>
Tags,O,O,O,O,O,O,O,O,O,O


In [63]:
def tag_text(text, tags, model, tokenizer):
  # 토큰 주비
  tokens = tokenizer(text).tokens()
  # 시퀀스를 입력 ID로 인코딩
  input_ids = xlmr_tokenizer(text, return_tensors='pt').input_ids.to(device)
  # 가능한 일곱 개의 클래스에 대한 로짓을 출력
  outputs = model(input_ids).logits
  # argmax 함수로 토큰마다 가장 가능성이 높은 클래스를 선택
  predictions = torch.argmax(outputs, dim=2)
  # 데이터프레임으로 변환
  preds = [tags.names[p] for p in predicitons[0].cpu().numpy()]
  return pd.DataFrame([tokens, preds], index=["Tokens","Tags"])

In [64]:
words, labels = de_example['tokens'], de_example['ner_tags_str']

In [65]:
tokenized_input = xlmr_tokenizer(de_example['tokens'], is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
pd.DataFrame([tokens], index=['Tokens'])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>


In [66]:
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index=['Tokens', 'Word IDs'])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
Word IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None


In [61]:
previous_word_idx = None

label_ids = []

for word_idx in word_ids:
  if word_idx is None or word_idx == previous_word_idx:
    label_ids.append(-100)
  elif word_idx != previous_word_idx:
    label_ids.append(labels[word_idx])
  previous_word_idx = word_idx

labels = [index2tag[l] if l != -100 else 'IGN' for l in label_ids]
index = ["Tokens", 'Word IDs', 'Label IDs', 'Labels']

pd.DataFrame([tokens, word_ids, label_ids, labels], index=index)

KeyError: 'O'